# 웹 스크래핑 프로그램 만들어보자
https://korean.visitkorea.or.kr/search/search_list.do?keyword=인천

1. 여행기사를 추출한다.
2. 검색어와 스크래핑 건수를 입력받는다.
3. 페이지를 이동한다.
4. 상세 페이지별로 여행기사 정보를 수집한다.
4. 이미지와 텍스트를 추출해서 파일로 저장

## my version

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import math
import time
import os
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time, urllib.request


options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=953,0')
options.add_experimental_option("detach", True)

search = input("검색어를 입력하세요: ")
URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search

driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(3)

검색어를 입력하세요: 인천


In [2]:
# 여행기사 더보기 클릭
driver.find_element(By.CSS_SELECTOR, '#s_recommend>.more_view>a').click()

In [3]:
# 수집할 데이터 개수 입력 받기
data_num = int(input('크롤링 할 건수는 몇 건? '))
num = 0

start_page = 1
goal_page = math.ceil(data_num / 10)
# 제목 추출 xpath
tit_path = '//*[@id="search_result"]/ul/li[*]/div[1]/div[1]/a'


data_list = []

for p in range(start_page, goal_page+1):
    print(f'===== {p} 페이지 =====')
    titles = driver.find_elements(By.XPATH, tit_path) # driver.find_element(By.CSS_SELECTOR, '#search_result .tit>a')
    time.sleep(1)
    
    for t in titles:
        num += 1
        if num > data_num:
            break
        title = t.text
        print(num, title)
        
        data_dic = {}
        data_dic['Title'] = title
        
        # 제목 클릭
        driver.execute_script('arguments[0].click()', t)
        
        
        # 상세정보 수집
        time.sleep(2)
        texts = driver.find_elements(By.CSS_SELECTOR, '.txt_p p')
        text_list = [ t.text for t in texts ]
        text_merge = ' '.join(text_list)
        data_dic['Text'] = text_merge
        data_list.append(data_dic)
        
        
        # 이미지 저장
        time.sleep(2)
        f_dir = f'여행기사/{search}/{num}'
        os.makedirs(os.getcwd() + '/' + f_dir, exist_ok=True)
        
        html_src = driver.page_source
        html_dom = BeautifulSoup(html_src, 'lxml')
        
        img_list = html_dom.select('.img_typeBox img')
        cdn_list = []
        for img in img_list:
            cdn_list.append(img["src"])
        no = 0
        for src in cdn_list:
            # 다운로드
            urllib.request.urlretrieve(src, f'{f_dir}/{no}.jpg')
            no+=1
        
        # back
        time.sleep(2)
        
        
        driver.back()
        
    if num < data_num:
        time.sleep(2)
        next_button = driver.find_element(By.CSS_SELECTOR, f"a[id='{p+1}']")
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)

print('========= 작업 완료 =========')

크롤링 할 건수는 몇 건? 12
===== 1 페이지 =====
1 서울 근교 여행, 아이와 나들이 떠나는 인천 여행 코스
2 옛 감성 물씬, 인천 구도심(동인천) 테마여행
3 인천테마여행_긴 물길을 따라 이어지는 다채로운 여행 경인아라뱃길
4 인천 여행, 선선한 저녁에 즐기는 인천 야경 명소 4
5 짜장면이 태어난 차이나타운의 먹자골목, 인천 북성동원조자장면거리와 짜장면박물관
6 인천 근현대사 중심지, 시민의 공간이 되다! 인천시민애집
7 낭만과 그리움을 찾아서, 인천 경인아라뱃길 정서진 드라이브
8 견공의, 견공에 의한, 견공을 위한 놀이 공간 인천대공원 반려견놀이터
9 [인천] 도심 속 공원으로 떠나는 비대면 힐링여행
인천 송도 센트럴파크 & 소래습지생태공원
10 우리나라 최초의 등대, 인천 팔미도등대
===== 2 페이지 =====
11 인천의 핫한 거리 평리단길 카페
12 과거부터 현재까지,
인천으로 떠나는 시간 여행
========= 작업 완료 =========


In [11]:
# import pandas as pd
# df = pd.DataFrame(data_list)
# df.to_csv(f'여행기사/{search}.csv', index=False, encoding='cp949') # 특정 문자에서 인코딩 문제 있음
# driver.close()

# pyinstaller로 실행 파일 만들기

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import math
import time
import os
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time, urllib.request
import pandas as pd

options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=953,0')
options.add_experimental_option("detach", True)

search = input("검색어를 입력하세요: ")
data_num = int(input('크롤링 할 건수는 몇 건? '))

URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search

driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(3)

# 여행기사 더보기 클릭
driver.find_element(By.CSS_SELECTOR, '#s_recommend>.more_view>a').click()

# 수집할 데이터 개수 입력 받기
num = 0

start_page = 1
goal_page = math.ceil(data_num / 10)
# 제목 추출 xpath
tit_path = '//*[@id="search_result"]/ul/li[*]/div[1]/div[1]/a'


data_list = []

for p in range(start_page, goal_page+1):
    print(f'===== {p} 페이지 =====')
    titles = driver.find_elements(By.XPATH, tit_path) # driver.find_element(By.CSS_SELECTOR, '#search_result')
    time.sleep(1)
    
    for t in titles:
        num += 1
        if num > data_num:
            break
        title = t.text
        print(num, title)
        
        data_dic = {}
        data_dic['Title'] = title
        
        # 제목 클릭
        driver.execute_script('arguments[0].click()', t)
        
        # 상세정보 수집
        time.sleep(2)
        texts = driver.find_elements(By.CSS_SELECTOR, '.txt_p p')
        text_list = [ t.text for t in texts ]
        text_merge = ' '.join(text_list)
        data_dic['Text'] = text_merge
        data_list.append(data_dic)
        
        # 이미지 저장
        time.sleep(2)
        f_dir = f'여행기사/{search}/{num}'
        os.makedirs(os.getcwd() + '/' + f_dir, exist_ok=True)
        
        html_src = driver.page_source
        html_dom = BeautifulSoup(html_src, 'lxml')
        
        img_list = html_dom.select('.img_typeBox img')
        cdn_list = []
        for img in img_list:
            cdn_list.append(img["src"])
        no = 0
        for src in cdn_list:
            # 다운로드
            urllib.request.urlretrieve(src, f'{f_dir}/{no}.jpg')
            no+=1
        
        # back
        time.sleep(2)
        driver.back()
        
    if num < data_num:
        time.sleep(2)
        next_button = driver.find_element(By.CSS_SELECTOR, f"a[id='{p+1}']")
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)

df = pd.DataFrame(data_list)
df.to_csv(f'여행기사/{search}.csv', index=False, encoding='cp949') # 특정 문자에서 인코딩 문제 있음
driver.close()
print('========= 작업 완료 =========')

검색어를 입력하세요:  부산
크롤링 할 건수는 몇 건?  2


===== 1 페이지 =====
1 부산 도심에서 가장 빨리 기장의 바다를 만나는 동해선
2 붉게 물든 노을이 아름다운, 부산 일몰 명소
========= 작업 완료 =========


,Title,Text
0,부산 도심에서 가장 빨리 기장의 바다를 만나는 동해선,"2016년 12월 동해선이 개통했다. 부전에서 일광까지 16개 역이 있으며, 총 2..."
1,"붉게 물든 노을이 아름다운, 부산 일몰 명소","출처 : 대한민국구석구석 SNS\n글, 사진 : 다님 3기 손창현\nhttps://..."
